# Linear Algebra - LU Decomposition

---

LU decomposition (or LU factorization) is a way of writing a square matrix $A$ with dimensions $n \times n$ as the product of a lower-triangular matrix and an upper-triangular matrix, more specifically:
$$
A = LU
$$
where 
* $L$ is a $n \times n$ as the lower triangular matrix with ones on the diagonal.
* $U$ is a $n \times n$ as the upper triangular matrix.

## Use Cases

**Ordinary Least Squares for Linear Regression**: 
$$ 
\underset{x}\argmin ||Ax - b||^2
$$ 
solves to 
$$ 
x = (A^\top A)^{-1} A^\top b
$$
Use $LU$ to solve
$$ 
(A^\top A) x = A^\top b
$$
by finding $LU = A^\top A$.

**Yule-Walker equations in AR time series models**, used to fit weights via autocorrelations:
$$
R \rho = r
$$
where $R$ is the autocorrelation matrix $R_{i,j} = \text{autocorrelation}(|i - j|)$, $\rho$ are the weights and $r$ is the vector of target autocorrelations.

## Doolittle Algorithm

For each column $k = 1, 2, \dots, n-1$:

1. Use $U_{kk}$ as the pivot in column $k$.

2. For each row $i = k+1, \dots, n$:

   a. Compute the multiplier
   $$
   L_{ik} = \frac{U_{ik}}{U_{kk}}.
   $$

   b. Eliminate the entry below the pivot by updating row $i$ of $U$:
   $$
   U_{ij} \leftarrow U_{ij} - L_{ik} \, U_{kj}
   \quad \text{for } j = k, k+1, \dots, n.
   $$

After this loop finishes for all $k$, the matrix $U$ is upper triangular and $L$ is lower triangular with ones on the diagonal, and they satisfy
$$
A = LU.
$$

## Example

Consider the matrix
$$
A = \begin{bmatrix}
2 & -1 & -2 \\
-4 & 6 & 3 \\
-4 & -2 & 8
\end{bmatrix}
$$

### First Column
Pivot: $u_{11} = 2$

Multipliers:
$$
L_{21} = -2, \quad L_{31} = -2
$$

Update rows 2 and 3:
$$
\text{Row 2} \leftarrow \text{Row 2} - (-2)\cdot\text{Row 1} = [0, 8, 7]
$$
$$
\text{Row 3} \leftarrow \text{Row 3} - (-2)\cdot\text{Row 1} = [0, 0, 4]
$$

Intermediate matrix:
$$
\begin{bmatrix}
2 & -1 & -2 \\
0 & 8 & 7 \\
0 & 0 & 4
\end{bmatrix}
$$

$L$ so far:
$$
L = \begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
-2 & 0 & 1
\end{bmatrix}
$$

### Second Column

Pivot: $u_{22} = 8$

Multiplier:
$$
L_{32} = 0
$$

No change needed. Final $U$:
$$
U = \begin{bmatrix}
2 & -1 & -2 \\
0 & 8 & 7 \\
0 & 0 & 4
\end{bmatrix}
$$

Final $L$:
$$
L = \begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
-2 & 0 & 1
\end{bmatrix}
$$

### Verification: $A = LU$

$$
LU = \begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
-2 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
2 & -1 & -2 \\
0 & 8 & 7 \\
0 & 0 & 4
\end{bmatrix}
= \begin{bmatrix}
2 & -1 & -2 \\
-4 & 6 & 3 \\
-4 & -2 & 8
\end{bmatrix} = A
$$

## Numerical Instability: $PA = LU$

Take 
$$
A = \begin{bmatrix} 
\epsilon & 1 \\ 
1 & 1 
\end{bmatrix}, \quad \epsilon = 10^{-16}
$$

Now with naive Doolittle, we have pivot = $\epsilon$. The multipliers are now huge:
$$
L_{21} = \frac{1}{\epsilon} = 10^{16}
$$

Then we have:
$$
\text{Row 2} \leftarrow \text{Row 2} - 10^{16} \times \text{Row 1}
$$

Precision issues can occur. Instead we can use **partial pivoting** (i.e. an additional matrix $P$) to scan column 1 and swap their rows:

$$
PA = \begin{bmatrix} 
0 & 1 \\ 
1 & 0 
\end{bmatrix}
\begin{bmatrix} 
\epsilon & 1 \\ 
1 & 1 
\end{bmatrix}
= \begin{bmatrix} 
1 & 1 \\ 
\epsilon & 1 
\end{bmatrix}
$$

Now $L_{21} = \epsilon = 10^{-16}$ which is safe. Then we have

$$
L = \begin{bmatrix} 1 & 0 \\ \epsilon & 1 \end{bmatrix}, \quad
U = \begin{bmatrix} 1 & 1 \\ 0 & 1 - \epsilon \end{bmatrix}
$$

Verify
$$
PA = \begin{bmatrix} 
0 & 1 \\ 
1 & 0 
\end{bmatrix}
\begin{bmatrix} 
\epsilon & 1 \\ 
1 & 1 
\end{bmatrix}
= \begin{bmatrix} 
1 & 1 \\ 
\epsilon & 1 
\end{bmatrix} 
= \begin{bmatrix} 1 & 0 \\ \epsilon & 1 \end{bmatrix} \begin{bmatrix} 1 & 1 \\ 0 & 1 - \epsilon \end{bmatrix} = LU
$$



## Solving $Ax = b$, given $PA = LU$

Permute $b$, i.e. compute $Pb$, then:
$$
\begin{align*}
PAx &= Pb \\
LUx &= Pb
\end{align*}
$$

Let $y = Ux$, then:

* Forward pass - solve for $y$: 
$$
Ly = Pb
$$
* Backward pass - solve for $x$:
$$
Ux = y
$$

## Example

Original system:
$$
A = \begin{bmatrix} \epsilon & 1 \\ 1 & 1 \end{bmatrix}, \quad
b = \begin{bmatrix} 3 \\ 5 \end{bmatrix}, \quad \epsilon = 10^{-16}
$$

From pivoting, we have:
$$
P = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}, \quad
L = \begin{bmatrix} 1 & 0 \\ \epsilon & 1 \end{bmatrix}, \quad
U = \begin{bmatrix} 1 & 1 \\ 0 & 1-\epsilon \end{bmatrix}
$$

Step 1: Apply permutation to b
$$
Pb = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} 3 \\ 5 \end{bmatrix}
   = \begin{bmatrix} 5 \\ 3 \end{bmatrix}
$$

Step 2: Forward substitution - Solve $Ly = Pb$
$$
L = \begin{bmatrix} 1 & 0 \\ \epsilon & 1 \end{bmatrix},
\quad Pb = \begin{bmatrix} 5 \\ 3 \end{bmatrix}
$$

$$
y_1 = 5
$$
$$
y_2 = 3 - \epsilon \cdot y_1 = 3 - \epsilon \cdot 5 \approx 3
$$

$$
y = \begin{bmatrix} 5 \\ 3 \end{bmatrix}
$$

Step 3: Back substitution - Solve $Ux = y$
$$
U = \begin{bmatrix} 1 & 1 \\ 0 & 1-\epsilon \end{bmatrix},
\quad y = \begin{bmatrix} 5 \\ 3 \end{bmatrix}
$$

$$
x_2 = \frac{y_2}{1-\epsilon} = \frac{3}{1-\epsilon} \approx 3
$$
$$
x_1 = \frac{y_1 - 1 \cdot x_2}{1} = 5 - 3 = 2
$$

**Final solution:**
$$
x = \begin{bmatrix} 2 \\ 3 \end{bmatrix}
$$

## Implementation

In [1]:
from collections import namedtuple

import numpy as np

from theoria.validor import TestCase, Validor

In [2]:
PLU = namedtuple("PLU", ["P", "L", "U"])

In [3]:
def lu_decompose(A: np.ndarray) -> PLU:
    """
    A LU decomposition with partial pivoting.

    Returns P, L, U such that PA = LU, where P is a permutation matrix,
    L is lower triangular with unit diagonal, and U is upper triangular.
    """

    A = np.array(A, dtype=float).copy()
    n = A.shape[0]
    P = np.eye(n)

    for i in range(n):
        # Find pivot
        pivot_row = i
        for k in range(i + 1, n):
            if abs(A[k, i]) > abs(A[pivot_row, i]):
                pivot_row = k

        # Swap
        if pivot_row != i:
            A[[i, pivot_row]] = A[[pivot_row, i]]
            P[[i, pivot_row]] = P[[pivot_row, i]]

        # CRITICAL: Skip elimination if no rows below
        if i == n - 1:
            continue

        for j in range(i + 1, n):
            if (
                abs(A[i, i]) > 1e-14
            ):  # Avoid division by zero, ... what to do otherwise?
                factor = A[j, i] / A[i, i]
                A[j, i] = factor
                for k in range(i + 1, n):
                    A[j, k] -= factor * A[i, k]

    L = np.tril(A, -1) + np.eye(n)
    U = np.triu(A)
    return PLU(P=P, L=L, U=U)


def lu_solve(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solve the linear system Ax = b using LU decomposition with partial pivoting.

    Returns the solution vector x.
    """
    plu = lu_decompose(A)
    P, L, U = plu.P, plu.L, plu.U
    Pb = P @ b

    # Forward substitution to solve Ly = Pb
    n = L.shape[0]
    y = np.zeros(n)
    for i in range(n):
        y[i] = Pb[i] - L[i, :i] @ y[:i]

    # Backward substitution to solve Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - U[i, i + 1 :] @ x[i + 1 :]) / U[i, i]

    return x

In [4]:
epsilon = 1e-9

decompose_test_cases = [
    TestCase(
        input_data={
            "A": np.array(
                [
                    [epsilon, 1.0],
                    [1.0, 1.0],
                ]
            )
        },
        expected_output=PLU(
            P=np.array(
                [
                    [0.0, 1.0],
                    [1.0, 0.0],
                ]
            ),
            L=np.array(
                [
                    [1.0, 0.0],
                    [epsilon, 1.0],
                ]
            ),
            U=np.array(
                [
                    [1.0, 1.0],
                    [0.0, 1 - epsilon],
                ],
            ),
        ),
        description="2x2 matrix with pivoting",
    ),
    TestCase(
        input_data={
            "A": np.array(
                [
                    [2.0, 1.0, -2.0],
                    [-4.0, 6.0, 3.0],
                    [-4.0, -2.0, 8.0],
                ]
            )
        },
        expected_output=PLU(
            P=np.array(
                [
                    [0.0, 1.0, 0.0],
                    [0.0, 0.0, 1.0],
                    [1.0, 0.0, 0.0],
                ]
            ),
            L=np.array(
                [
                    [1.0, 0.0, 0.0],
                    [1.0, 1.0, 0.0],
                    [-0.5, -0.5, 1.0],
                ]
            ),
            U=np.array(
                [
                    [-4.0, 6.0, 3.0],
                    [0.0, -8.0, 5.0],
                    [0.0, 0.0, 2.0],
                ],
            ),
        ),
        description="3x3 matrix with pivoting",
    ),
]


Validor(lu_decompose).add_cases(decompose_test_cases).run(
    lambda result, expected: all(
        [
            np.allclose(result.P, expected.P),
            np.allclose(result.L, expected.L),
            np.allclose(result.U, expected.U),
        ]
    )
)

[2026-01-16 12:28:05,087] [INFO] All 2 tests passed for lu_decompose.


In [5]:
def decompose_multiply_test(A):
    plu = lu_decompose(A)
    PA = plu.P @ A
    LU = plu.L @ plu.U
    return np.allclose(PA, LU)


decompose_multiply_test_cases = [
    TestCase(
        input_data={
            "A": np.array(
                [
                    [2.0, 1.0, -2.0],
                    [-4.0, 6.0, 3.0],
                    [-4.09, -2.0, 8.0],
                ]
            )
        },
        expected_output=True,
        description="3x3 matrix PA=LU verification",
    ),
    TestCase(
        input_data={
            "A": np.array(
                [
                    [1.0, 2.0, 3.0, 4.0],
                    [0.0, 1.0, 4.0, 5.0],
                    [5.0, 6.0, 0.0, -1.0],
                    [7.0, 8.0, 9.0, 10.0],
                ]
            )
        },
        expected_output=True,
        description="4x4 matrix PA=LU",
    ),
]


Validor(decompose_multiply_test).add_cases(decompose_multiply_test_cases).run()

[2026-01-16 12:28:05,102] [INFO] All 2 tests passed for decompose_multiply_test.


In [6]:
solve_test_cases = [
    TestCase(
        input_data={
            "A": np.array(
                [
                    [1.0, 0.0],
                    [0.0, 1.0],
                ]
            ),
            "b": np.array([1.0, 2.0]),
        },
        expected_output=np.array([1.0, 2.0]),
        description="Solve identity matrix system",
    ),
    TestCase(
        input_data={
            "A": np.array(
                [
                    [2.0, 1.0, 1.0],
                    [4.0, -6.0, 0.0],
                    [-2.0, 7.0, 2.0],
                ]
            ),
            "b": np.array([1.0, 2.0, 3.0]),
        },
        expected_output=np.array([-1.0, -1.0, 4.0]),
        description="Solve 3x3 system",
    ),
]

Validor(lu_solve).add_cases(solve_test_cases).run(
    lambda result, expected: np.allclose(result, expected)
)

[2026-01-16 12:28:05,114] [INFO] All 2 tests passed for lu_solve.


## Scipy Modules

In [7]:
from scipy.linalg import lu


def scipy_decompose_test(A):
    plu = lu(A)
    P, L, U = plu
    return np.allclose(P @ A, L @ U)


Validor(scipy_decompose_test).add_cases(decompose_multiply_test_cases).run()

[2026-01-16 12:28:05,256] [INFO] All 2 tests passed for scipy_decompose_test.


In [8]:
from scipy.linalg import solve


def scipy_solve(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    return solve(A, b)


Validor(scipy_solve).add_cases(solve_test_cases).run(
    lambda result, expected: np.allclose(result, expected)
)

[2026-01-16 12:28:05,270] [INFO] All 2 tests passed for scipy_solve.
